# Importing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../')
from hydra import initialize, compose, initialize_config_module
from src.dataset.data_module import DataModule

In [4]:
import ee
import geemap
import fiona
import time
import json
from pathlib import Path
import datetime
import numpy as np
from src.utils.gee import export_sar, export_opt, export_maskcloud, export_opt_l1
import yaml

In [6]:
site = 's3'
with initialize(version_base=None, config_path='../conf'):
    cfg = compose(config_name='config.yaml', 
                  overrides=[
                      f"+site={site}", 
                      "+exp=exp_1"
                      ])

# Authentication

In [7]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


# Site Selection

In [8]:
cfg

{'path': {'base': '/home/felferrari/projects/thesis2', 'original_data': '${.base}/data/original', 'opt': '${.original_data}/opt', 'sar': '${.original_data}/sar', 'prodes': {'base': '${..original_data}/prodes', 'yearly_deforestation': '${.base}/yearly_deforestation.shp', 'previous_deforestation': '${.base}/accumulated_deforestation_2000.shp', 'no_forest': '${.base}/no_forest.shp', 'residual': '${.base}/residual.shp', 'hydrography': '${.base}/hydrography.shp'}, 'prepared': {'base': '${..base}/data/prepared', 'train': '${.base}/train', 'validation': '${.base}/validation', 'opt_statistics': '${.base}/opt_stats.csv', 'sar_statistics': '${.base}/sar_stats.csv'}, 'general': '${.base}/data/general', 'label': {'train': '${..general}/label_train.tif', 'test': '${..general}/label_test.tif'}, 'prev_map': {'train': '${..general}/previous_train.tif', 'test': '${..general}/previous_test.tif'}, 'tiles': '${.general}/tiles.tif', 'train_val_map': '${.general}/train_val.tif'}, 'preparation': {'generate':

# Reading Location data

In [9]:
roi = ee.Geometry.Rectangle([cfg.site['xmin'], cfg.site['ymin'], cfg.site['xmax'], cfg.site['ymax']])

In [10]:
map_view = geemap.Map(center=(cfg.site['ycenter'], cfg.site['xcenter']), zoom=8)
map_view


Map(center=[-4.493, -43.203], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…

In [11]:
#location = geemap.shp_to_ee(str(location_shape))
map_view.addLayer(roi, {}, 'Location')

# Dates

In [20]:
before_days = cfg['before_days']
d0 = datetime.datetime.strptime(cfg['date'][0], "%Y-%m-%d")
d1 = datetime.datetime.strptime(cfg['date'][1], "%Y-%m-%d")
d2 = datetime.datetime.strptime(cfg['date'][2], "%Y-%m-%d")
dt1 = (d1-d0).days + before_days
dt2 = (d2-d1).days + before_days

In [21]:
start_date = d0 - datetime.timedelta(days=before_days)
end_date = d2 + datetime.timedelta(days=before_days)
print(f'Analyzed Period: {start_date.date()}:{end_date.date()}')


Analyzed Period: 2019-08-09:2021-09-15


In [22]:
opt_vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 2048
}

sar_vizParams = {
  'bands': ['VV'],
  'min': 0,
  'max': 1
}

cloud_vizParams = {
  'bands': ['MSK_CLDPRB'],
  'min': 0,
  'max': 100
}

# SAR Data

## Average Data

In [30]:
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(start_date.date()), str(end_date.date())).filterBounds(roi)
ds = np.arange(dt1)
img_collections = []
split_ds = np.array_split(ds, cfg['n_sar_avg_images'])
for i, dsi in enumerate(split_ds):
    di0 = d0 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[0]))
    di1 = d0 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[-1]))
    img_col = sar_collection.filterDate(str(di0.date()), str(di1.date()))
    img_col = img_col.filter(ee.Filter.eq('platform_number', 'A'))
    img = img_col.mean().clip(roi)
    #print(f'{di0.date()}-{di1.date()}-delta:{(di1-di0).days}|{len(img_col.getInfo()["features"])} images')
    print(f'{di0.date()}-{di1.date()}.tif')
    map_view.addLayer(img, sar_vizParams, f't1-{i:02d}')
    export_sar(img, roi, f'{di0.date()}-{di1.date()}')
    
    
    

2019-08-09-2019-09-08.tif
2019-09-09-2019-10-09.tif
2019-10-10-2019-11-09.tif
2019-11-10-2019-12-10.tif
2019-12-11-2020-01-10.tif
2020-01-11-2020-02-10.tif
2020-02-11-2020-03-12.tif
2020-03-13-2020-04-11.tif
2020-04-12-2020-05-11.tif
2020-05-12-2020-06-10.tif
2020-06-11-2020-07-10.tif
2020-07-11-2020-08-09.tif


In [31]:
ds = np.arange(dt2)
img_collections = []
split_ds = np.array_split(ds,  cfg['n_sar_avg_images'])
for i, dsi in enumerate(split_ds):
    di0 = d1 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[0]))
    di1 = d1 - datetime.timedelta(days=before_days) + datetime.timedelta(days=int(dsi[-1]))
    img_col = sar_collection.filterDate(str(di0.date()), str(di1.date()))
    img_col = img_col.filter(ee.Filter.eq('platform_number', 'A'))
    img = img_col.mean().clip(roi)
    #print(f'{di0.date()}-{di1.date()}-delta:{(di1-di0).days}|{len(img_col.getInfo()["features"])} images')
    print(f'{di0.date()}-{di1.date()}.tif')
    #map_view.addLayer(img, sar_vizParams, f't1-{i:02d}')
    export_sar(img, roi, f'{di0.date()}-{di1.date()}')

2020-07-10-2020-08-12.tif
2020-08-13-2020-09-15.tif
2020-09-16-2020-10-19.tif
2020-10-20-2020-11-22.tif
2020-11-23-2020-12-26.tif
2020-12-27-2021-01-28.tif
2021-01-29-2021-03-02.tif
2021-03-03-2021-04-04.tif
2021-04-05-2021-05-07.tif
2021-05-08-2021-06-09.tif
2021-06-10-2021-07-12.tif
2021-07-13-2021-08-14.tif


## Multiple Images

In [39]:
date_0, date_1 = d0 - datetime.timedelta(days=12), d1 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
#skip_idx = []
skip_idx = np.arange(len(cycles))[3:-4]
count = 0
for idx, cycle in enumerate(cycles):
    if idx in skip_idx:
        continue
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.mosaic()
    data_check = img.mask().reduceRegion(
        ee.Reducer.min(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    if data_check['angle'] == 1:
        count+=1
        img = img.clip(roi)
        map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: s_{dates[0].date()}-{dates[-1].date()}.tif')
        export_sar(img, roi, f's_{dates[0].date()}-{dates[-1].date()}')
print(count)

130
2019-07-12---2020-08-07
33
0: s_2019-07-14-2019-07-21.tif
1: s_2019-07-26-2019-08-02.tif
2: s_2019-08-07-2019-08-14.tif
29: s_2020-06-26-2020-07-03.tif
30: s_2020-07-08-2020-07-15.tif
31: s_2020-07-20-2020-07-27.tif
6


In [40]:
date_0, date_1 = d1 - datetime.timedelta(days=12), d2 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
#skip_idx = []
skip_idx = np.arange(len(cycles))[4:-4]
count = 0
for idx, cycle in enumerate(cycles):
    if idx in skip_idx:
        continue
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.mosaic()
    data_check = img.mask().reduceRegion(
        ee.Reducer.min(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    if data_check['angle'] == 1:
        count+=1
        img = img.clip(roi)
        #map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: s_{dates[0].date()}-{dates[-1].date()}.tif')
        export_sar(img, roi, f's_{dates[0].date()}-{dates[-1].date()}')
print(count)

132
2020-07-14---2021-08-10
34
1: s_2020-07-20-2020-07-27.tif
2: s_2020-08-01-2020-08-08.tif
3: s_2020-08-13-2020-08-20.tif
30: s_2021-07-03-2021-07-10.tif
31: s_2021-07-15-2021-07-22.tif
32: s_2021-07-27-2021-08-03.tif
6


## Log data 2 dates

In [ ]:
date_0, date_1 = d0 - datetime.timedelta(days=12), d1 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
cycles = [cycle for i, cycle in enumerate(cycles) if i == 0 or i == (len(cycles)-2) ]
for idx, cycle in enumerate(cycles):
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.median()
    data_check = img.mask().reduceRegion(
        ee.Reducer.mean(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    map_view.addLayer(img.clip(roi), sar_vizParams, f't2-{str(cycle)}')
    if data_check['VH'] >= 0.999:
        img = img.clip(roi).float()
        map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: log_{dates[0].date()}-{dates[-1].date()}.tif')
        export_sar(img, roi, f'log_{dates[0].date()}-{dates[-1].date()}')

130
2019-07-12---2020-08-07
33
0: log_2019-07-14-2019-07-21.tif
1: log_2020-07-20-2020-07-27.tif


In [ ]:
date_0, date_1 = d1 - datetime.timedelta(days=12), d2 + datetime.timedelta(days=12)
sar_collection = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(str(date_0.date()), str(date_1.date())).filterBounds(roi)
img_col = sar_collection.filter(ee.Filter.eq('platform_number', 'A'))
imgs = img_col.getInfo()['features']
print(len(imgs))
print(f'{date_0.date()}---{date_1.date()}')
cycles = set({})
for im in imgs:
    cycle = im['properties']['cycleNumber'] 
    cycles.add(cycle)
cycles = list(cycles)
print(len(cycles))
cycles = [cycle for i, cycle in enumerate(cycles) if i == 1 or i == (len(cycles)-2) ]
for idx, cycle in enumerate(cycles):
    cycle_col = sar_collection.filter(ee.Filter.eq('cycleNumber', cycle))
    img = cycle_col.median()
    data_check = img.mask().reduceRegion(
        ee.Reducer.mean(), 
        roi,
        scale = 10,
        maxPixels = 1e9).getInfo()
    #if not 0 in data_check.values():
    map_view.addLayer(img.clip(roi), sar_vizParams, f't2-{str(cycle)}')
    if data_check['VH'] >= 0.999:
        img = img.clip(roi).float()
        map_view.addLayer(img, sar_vizParams, f't2-{str(cycle)}')
        dates = set({})
        cycle_imgs = cycle_col.getInfo()["features"]
        for cycle_img in cycle_imgs:
            day = datetime.datetime.strptime(cycle_img['properties']['system:index'][17:25], '%Y%m%d')
            dates.add(day)
        dates = list(dates)
        dates.sort()
        print(f'{idx}: log_{dates[0].date()}-{dates[-1].date()}.tif')
        export_sar(img, roi, f'log_{dates[0].date()}-{dates[-1].date()}')

132
2020-07-14---2021-08-10
34
0: log_2020-07-20-2020-07-27.tif
1: log_2021-07-27-2021-08-03.tif


# Optical Data (Level 2A)

In [23]:
opt_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate(start_date, end_date)
opt_collection = opt_collection.filterBounds(roi)
opt_info = opt_collection.getInfo()

In [14]:
for info in opt_info['features']:
    img_date = datetime.datetime.strptime(info['properties']['DATATAKE_IDENTIFIER'][5:13], '%Y%m%d')
    img_lct = info['properties']['PRODUCT_ID']
    cloud = info['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print(img_date, img_lct, cloud)
    #break

2019-08-10 00:00:00 S2A_MSIL2A_20190810T131251_N0213_R138_T23MQQ_20190810T152632 0.028127
2019-08-10 00:00:00 S2A_MSIL2A_20190810T131251_N0213_R138_T23MQR_20190810T152632 0.017341
2019-08-13 00:00:00 S2A_MSIL2A_20190813T132241_N0213_R038_T23MPQ_20190813T154307 9.009592
2019-08-13 00:00:00 S2A_MSIL2A_20190813T132241_N0213_R038_T23MPR_20190813T154307 30.186472
2019-08-13 00:00:00 S2A_MSIL2A_20190813T132241_N0213_R038_T23MQQ_20190813T154307 2.095576
2019-08-13 00:00:00 S2A_MSIL2A_20190813T132241_N0213_R038_T23MQR_20190813T154307 23.158098
2019-08-15 00:00:00 S2B_MSIL2A_20190815T131249_N0213_R138_T23MQQ_20190815T151734 39.361535
2019-08-15 00:00:00 S2B_MSIL2A_20190815T131249_N0213_R138_T23MQR_20190815T151734 32.437139
2019-08-15 00:00:00 S2B_MSIL2A_20190815T131249_N0213_R138_T23MQQ_20190815T171203 29.295124
2019-08-18 00:00:00 S2B_MSIL2A_20190818T132239_N0213_R038_T23MPQ_20190818T144102 0.119708
2019-08-18 00:00:00 S2B_MSIL2A_20190818T132239_N0213_R038_T23MPR_20190818T144102 3.668677
2019-

In [37]:
for date_i in cfg['opt_images_dates']:
    opt_d0 = date_i[0]
    opt_d1 = date_i[1]
    opt_collection_i = opt_collection.filterDate(opt_d0, opt_d1)
    opt_mosaic_i = opt_collection_i.mosaic()
    export_opt(opt_mosaic_i, roi, f'{opt_d0}-{opt_d1}')
    export_maskcloud(opt_mosaic_i, roi, f'cloud_{opt_d0}-{opt_d1}')
    #map_view.addLayer(opt_mosaic_i.clip(roi), opt_vizParams, f'opt-{opt_d0}-{opt_d1}')
    print(f'{opt_d0}-{opt_d1}.tif')
    #print(f'{len(opt_collection_i.getInfo()["features"])} images')

2019-09-27-2019-09-28.tif


# Optical Data (Level 1C)

In [29]:
opt_collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(start_date, end_date)
opt_collection = opt_collection.filterBounds(roi)
opt_info = opt_collection.getInfo()

In [30]:
for info in opt_info['features']:
    img_date = datetime.datetime.strptime(info['properties']['DATATAKE_IDENTIFIER'][5:13], '%Y%m%d')
    img_lct = info['properties']['PRODUCT_ID']
    cloud = info['properties']['CLOUDY_PIXEL_PERCENTAGE']
    print(img_date, img_lct, cloud)
    #break

2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MXM_20190626T154048 0.1723
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MXN_20190626T154048 2.4004
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MYM_20190626T154048 0
2019-06-26 00:00:00 S2A_MSIL1C_20190626T140101_N0207_R067_T21MYN_20190626T154048 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MXM_20190701T154033 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MXN_20190701T154033 0.0128
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MYM_20190701T154033 0
2019-07-01 00:00:00 S2B_MSIL1C_20190701T140059_N0207_R067_T21MYN_20190701T154033 0
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MXM_20190706T153903 45.7992
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MXN_20190706T153903 70.2141
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R067_T21MYM_20190706T153903 14.0007
2019-07-06 00:00:00 S2A_MSIL1C_20190706T140101_N0207_R

In [31]:
for date_i in cfg['opt_images_dates']:
    opt_d0 = date_i[0]
    opt_d1 = date_i[1]
    opt_collection_i = opt_collection.filterDate(opt_d0, opt_d1)
    opt_mosaic_i = opt_collection_i.mosaic()
    export_opt_l1(opt_mosaic_i, roi, f'{opt_d0}-{opt_d1}')
    map_view.addLayer(opt_mosaic_i.clip(roi), opt_vizParams, f'opt-{opt_d0}-{opt_d1}')
    print(f'{opt_d0}-{opt_d1}.tif')
    #print(f'{len(opt_collection_i.getInfo()["features"])} images')    

2021-07-25-2021-07-26.tif
2021-08-04-2021-08-05.tif
